In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd  # Import Pandas

# Initialize the Firefox WebDriver (adjust the executable_path)
driver = webdriver.Firefox()

# Define the base URL
base_url = "https://app.alt.xyz/browse/liquid-auctions?sortBy=highest_alt_value_first"

# Specify the number of pages you want to scrape
num_pages = 1  # Adjust as needed

# Initialize lists to store scraped data separately
card_data = []
player_data = []
card_number_data = []
grade_data = []
price_data = []
link_data = []  # Initialize a list to store links

# Loop through the pages
for page in range(1, num_pages + 1):
    # Construct the URL for the current page
    current_url = f"{base_url}&page={page}"
    
    # Navigate to the current page
    driver.get(current_url)
    
    # Add your scraping logic here
    # Locate div elements with class 'css-12t1va8' and 'css-wbtmg0' and extract data
    
    div_elements = driver.find_elements(By.CLASS_NAME, 'css-12t1va8') + driver.find_elements(By.CLASS_NAME, 'css-wbtmg0')
    
    for div_element in div_elements:
        try:
            # Re-locate span elements within the div
            span_elements = div_element.find_elements(By.TAG_NAME, 'span')
            
            # Check if there are three span elements in 'css-12t1va8' div
            if div_element.get_attribute('class') == 'css-12t1va8' and len(span_elements) == 3:
                card_data.append(span_elements[0].text)
                # Check if the second span's text contains '#'
                if '#' in span_elements[1].text:
                    player, card_number = span_elements[1].text.split('#', 1)
                    player_data.append(player.strip())  # Remove extra whitespace
                    card_number_data.append(card_number.strip())  # Remove extra whitespace
                else:
                    player_data.append(span_elements[1].text.strip())
                    card_number_data.append('')
                grade_data.append(span_elements[2].text)
            # Check if there is a 'css-wbtmg0' div with a span having the specified classes
            elif div_element.get_attribute('class') == 'css-wbtmg0' and len(span_elements) == 1 and \
                 'MuiTypography-root' in span_elements[0].get_attribute('class') and \
                 'MuiTypography-vegaH7' in span_elements[0].get_attribute('class') and \
                 'css-yrl21b' in span_elements[0].get_attribute('class'):
                price_data.append(span_elements[0].text)
            
            # Use WebDriverWait to wait for the element with class 'css-dcq7s5'
            link_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'css-dcq7s5'))
            )
            
            # Extract the link's href attribute and store it in the list
            link_data.append(link_element.get_attribute('href'))
            
        except Exception as e:
            print(f"An error occurred: {e}")
    
    # Scroll down to trigger JavaScript loading (you may need to adjust the amount)
    actions = ActionChains(driver)
    actions.send_keys(Keys.END).perform()
    
    # Add a delay to allow JavaScript content to load
    time.sleep(5)  # Adjust the wait time as needed

# Close the WebDriver when finished
driver.quit()

# Ensure all lists have the same length by filling missing data with empty strings
min_length = min(len(card_data), len(player_data), len(card_number_data), len(grade_data), len(price_data), len(link_data))
card_data = card_data[:min_length]
player_data = player_data[:min_length]
card_number_data = card_number_data[:min_length]
grade_data = grade_data[:min_length]
price_data = price_data[:min_length]
link_data = link_data[:min_length]

# Create a DataFrame from the scraped data
data = {
    'Card': card_data,
    'Player': player_data,
    'Card Number': card_number_data,
    'Grade': grade_data,
    'Price': price_data,
    'Link': link_data  # Add the list of links to the DataFrame
}

df = pd.DataFrame(data)

# Specify the path to the CSV file where you want to save the data
csv_file_path = 'scraped_data.csv'

# Write the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

# Confirm that the data has been saved to the CSV file
print(f"Data has been saved to {csv_file_path}")

Data has been saved to scraped_data.csv


In [2]:
df

,Card,Player,Card Number,Grade,Price,Link
0,1909 T206 Sweet Caporal,Ty Cobb/Portrait Green,97,PSA 8,"$450,000",https://app.alt.xyz/browse/external-listing?id...
1,1909 T206 Sweet Caporal,Ty Cobb/Portrait Green,97,PSA 8,"$450,000",https://app.alt.xyz/browse/external-listing?id...
2,1965 Topps,Joe Namath,122,PSA 8.5,"$102,000",https://app.alt.xyz/browse/external-listing?id...
3,1965 Topps,Joe Namath,122,PSA 8.5,"$108,000",https://app.alt.xyz/browse/external-listing?id...
4,1952 Topps,Mickey Mantle,311,PSA 4,"$34,800",https://app.alt.xyz/browse/external-listing?id...
5,1952 Topps,Mickey Mantle,311,PSA 4,"$120,000",https://app.alt.xyz/browse/external-listing?id...
6,1968 Topps,Jerry Koosman/Nolan Ryan,177,PSA 9,"$34,800",https://app.alt.xyz/browse/external-listing?id...
7,1998 Hoops Slam Bams,Michael Jordan,1,PSA 9,"$120,000",https://app.alt.xyz/browse/external-listing?id...
8,1968 Topps,Jerry Koosman/Nolan Ryan,177,PSA 9,"$99,466",https://app.alt.xyz/browse/external-listing?id...
9,1952 Topps,Mickey Mantle,311,PSA 2,"$99,466",https://app.alt.xyz/browse/external-listing?id...


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Set up the WebDriver
driver = webdriver.Firefox(executable_path='./geckodriver')

# Initialize an empty DataFrame to store the data
result_df = pd.DataFrame(columns=['Links'])

# Define the URL pattern for pagination
base_url = 'https://app.alt.xyz/browse/liquid-auctions?sortBy=highest_alt_value_first&page={}'  # Replace with your URL

# Define the number of pages you want to scrape
num_pages = 1 # Adjust as needed

try:
    for page in range(1, num_pages + 1):
        # Generate the URL for the current page
        url = base_url.format(page)
        
        # Navigate to the URL
        driver.get(url)

        # Wait for anchor tags with class "css-dcq7s5" to be visible (increased timeout)
        WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, '.css-dcq7s5')))

        # Find all anchor tags with class "css-dcq7s5" and extract their href attribute (link)
        anchor_tags = driver.find_elements(By.CSS_SELECTOR, '.css-dcq7s5')

        # Extract links and store them in a list
        links = [anchor.get_attribute('href') for anchor in anchor_tags]

        # Create a DataFrame for the current page's links
        page_df = pd.DataFrame({'Links': links})

        # Append the page's data to the result DataFrame
        result_df = pd.concat([result_df, page_df], ignore_index=True)

except Exception as e:
    print(f"An error occurred: {str(e)}")

finally:
    # Close the WebDriver
    driver.quit()

# Print the final result DataFrame
result_df

#############################################333

# Set up the WebDriver
driver = webdriver.Firefox(executable_path='./geckodriver')

# Load your existing DataFrame with links
# Replace this with your actual DataFrame creation or loading code
# For this example, let's assume you have a DataFrame df with a 'Links' column
# df = pd.read_csv('your_data.csv')  # Load your DataFrame from a CSV file, for instance

# Create empty lists to store the highest bids, number of bids, and additional links
highest_bids = []
num_bids_list = []  # Rename this variable to num_bids_list
additional_links = []  # Store the additional links here

# Loop through the DataFrame and open each link
for index, row in result_df.iterrows():
    link = row['Links']  # Assuming 'Links' is the column name containing the URLs
    
    # Navigate to the URL
    driver.get(link)
    
    # You can perform further actions on the webpage here, if needed
    
    # Find the <span> tag with multiple classes for highest bid
    try:
        highest_bid_element = driver.find_element_by_css_selector('span.MuiTypography-root.MuiTypography-vegaH5.css-gowc6z')
        highest_bid = highest_bid_element.text
    except Exception as e:
        highest_bid = 'N/A'  # Handle cases where the span tag for highest bid is not found or there's an error
    
    # Find and collect the second "Number of Bids" element in the same <span>
    try:
        num_bids_elements = driver.find_elements_by_css_selector('span.MuiTypography-root.MuiTypography-vegaH5.css-gowc6z')
        num_bids = num_bids_elements[1].text if len(num_bids_elements) > 1 else 'N/A'
    except Exception as e:
        num_bids = 'N/A'  # Handle cases where the "Number of Bids" element is not found or there's an error
    
    # Find and collect the additional link (assuming it's in an anchor tag)
    try:
        additional_link_element = driver.find_element_by_css_selector('a.additional-link-selector')  # Replace with your selector
        additional_link = additional_link_element.get_attribute('href')
    except Exception as e:
        additional_link = 'N/A'  # Handle cases where the additional link is not found or there's an error
    
    # Append the highest bid, number of bids, and additional link to their respective lists
    highest_bids.append(highest_bid)
    num_bids_list.append(num_bids)
    additional_links.append(additional_link)
    
# Add the highest bids, number of bids, and additional links to the DataFrame
result_df['HighestBid'] = highest_bids
result_df['NumberOfBids'] = num_bids_list
result_df['AdditionalLink'] = additional_links  # Add the additional links

# Close the WebDriver when done
driver.quit()

# Print the updated DataFrame
result_df
####################################

# Assuming you already have a DataFrame called result_df with a column "Links"

# Initialize the Firefox web driver
driver = webdriver.Firefox()

try:
    # Create an empty list to store dictionaries containing the link, collected text, and best offer price
    collected_data = []
    
    # Iterate through the URLs in the DataFrame
    for url in result_df['Links']:
        try:
            # Open the URL in the Firefox browser
            driver.get(url)
            
            # You may need to wait for the page to fully load
            # You can wait for a specific element to appear or use a time delay
            # For example, to wait for 10 seconds:
            driver.implicitly_wait(10)
            
            # Find the <span> element with the specified class names for collected text
            span_elements = driver.find_elements_by_css_selector("span.MuiTypography-root.MuiTypography-vegaBody3.css-11ntl8z")
            
            # Find the <span> element with the specified class names for best offer price
            best_offer_price_elements = driver.find_elements_by_css_selector("span.MuiTypography-root.MuiTypography-vegaSubtitle2.css-16tlq5a")
            
            # Extract and collect the text for collected text and best offer price
            collected_texts = [span.text for span in span_elements]
            best_offer_prices = [price.text for price in best_offer_price_elements]
            
            # Create a dictionary with the link, collected text, and best offer price
            for i in range(len(collected_texts)):
                collected_data.append({
                    'Link': url,
                    'Collected_Text': collected_texts[i],
                    'Best_Offer_Price': best_offer_prices[i]
                })
            
            # Customize the code to extract and process the specific data you need
            
        except Exception as e:
            print(f"Failed to retrieve data from {url}: {str(e)}")

    # Create a new DataFrame for the collected data
    collected_df = pd.DataFrame(collected_data)
    
finally:
    # Close the Firefox web driver when you're done
    driver.quit()

# Display the collected data DataFrame
collected_df
#######################3
merged_df = pd.merge(result_df, collected_df, left_on='Links', right_on='Link', how='outer')

# Drop the duplicate "Link" column (if needed)
merged_df.drop(columns=['Link'], inplace=True)
#merged_df.drop(columns=['Collected_Texts'], inplace=True)
merged_df
####################
# Assuming you have the merged DataFrame named merged_df

# Remove dollar signs ($) and commas (,) and convert "Best_Offer_Price" to a numeric type
merged_df['Best_Offer_Price'] = merged_df['Best_Offer_Price'].str.replace('$', '').str.replace(',', '').astype(float)

# Group the DataFrame by the "Links" column
grouped = merged_df.groupby('Links')

# Calculate the mean, median, average, and max for "Best_Offer_Price" in each group
result_summary = grouped['Best_Offer_Price'].agg(['mean', 'median', 'mean', 'max'])

# Rename the columns for clarity
result_summary.columns = ['Mean', 'Median', 'Average', 'Max']

# Merge the summary statistics back into the original DataFrame
merged_df = pd.merge(merged_df, result_summary, left_on='Links', right_index=True)

# Display the updated DataFrame
merged_df
########333333333
# Calculate the breakeven values based on the specified conditions
merged_df['breakeven'] = merged_df['Median'].apply(lambda median: median * 0.62 if median < 2500 else 1560 + (median - 2500) * 0.7)
merged_df['10pc'] = merged_df['Median'].apply(lambda median: median * 0.56 if median < 2500 else 1400 + (median - 2500) * 0.63)
# Calculate the "$unser" values based on the specified formula
merged_df['$under'] = merged_df['breakeven'] - (merged_df['Best_Offer_Price'] / 1.2)
# Calculate the "pcnt_under" values based on the specified formula
merged_df['pcnt_under'] = merged_df['Best_Offer_Price'] / 1.2 / merged_df['breakeven']
# Display the updated DataFrame
merged_df


Failed to retrieve data from https://app.alt.xyz/browse/external-listing?id=3fc7c20e-d965-42c9-9d5c-b2dd153607b3&prev=%2Fbrowse%2Fliquid-auctions%3FsortBy%3Dhighest_alt_value_first: list index out of range
Failed to retrieve data from https://app.alt.xyz/browse/external-listing?id=e0c6abc7-2e62-49da-b8d5-2757fe9676e0&prev=%2Fbrowse%2Fliquid-auctions%3FsortBy%3Dhighest_alt_value_first: list index out of range
Failed to retrieve data from https://app.alt.xyz/browse/external-listing?id=536313fc-2e76-4d81-bf53-3eac18c10ffc&prev=%2Fbrowse%2Fliquid-auctions%3FsortBy%3Dhighest_alt_value_first: list index out of range
Failed to retrieve data from https://app.alt.xyz/browse/external-listing?id=2c5e5300-915e-44b7-a321-8060e86fe3c6&prev=%2Fbrowse%2Fliquid-auctions%3FsortBy%3Dhighest_alt_value_first: list index out of range
Failed to retrieve data from https://app.alt.xyz/browse/external-listing?id=fe3ba824-e104-4318-b723-b1d067baffca&prev=%2Fbrowse%2Fliquid-auctions%3FsortBy%3Dhighest_alt_value_f

C:\Users\neste\AppData\Local\Temp\ipykernel_15524\1043844307.py:177: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged_df['Best_Offer_Price'] = merged_df['Best_Offer_Price'].str.replace('$', '').str.replace(',', '').astype(float)


,Links,HighestBid,NumberOfBids,AdditionalLink,Collected_Text,Best_Offer_Price,Mean,Median,Average,Max,breakeven,10pc,$under,pcnt_under
0,https://app.alt.xyz/browse/external-listing?id...,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://app.alt.xyz/browse/external-listing?id...,"$450,000",19,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://app.alt.xyz/browse/external-listing?id...,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://app.alt.xyz/browse/external-listing?id...,"$108,000",8,N/A,"Aug 24, 2023",252000.0,586500.0,342000.0,586500.0,1410000.0,239210.0,215285.00,29210.000000,0.877890
4,https://app.alt.xyz/browse/external-listing?id...,"$108,000",8,N/A,Auto 10,324000.0,586500.0,342000.0,586500.0,1410000.0,239210.0,215285.00,-30790.000000,1.128715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,https://app.alt.xyz/browse/external-listing?id...,"$15,600",4,N/A,Auto 10,38100.0,32101.0,32101.0,32101.0,38100.0,22280.7,20048.63,-9469.300000,1.425000
87,https://app.alt.xyz/browse/external-listing?id...,"$25,200",18,N/A,"Jul 21, 2022",50400.0,50400.0,50400.0,50400.0,50400.0,35090.0,31577.00,-6910.000000,1.196922
88,https://app.alt.xyz/browse/external-listing?id...,"$20,400",8,N/A,"Feb 20, 2018",7600.0,7600.0,7600.0,7600.0,7600.0,5130.0,4613.00,-1203.333333,1.234568
89,https://app.alt.xyz/browse/external-listing?id...,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
merged_df

,Links,HighestBid,NumberOfBids,AdditionalLink,Collected_Text,Best_Offer_Price,Mean,Median,Average,Max,breakeven,10pc,$under,pcnt_under
0,https://app.alt.xyz/browse/external-listing?id...,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://app.alt.xyz/browse/external-listing?id...,"$450,000",19,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://app.alt.xyz/browse/external-listing?id...,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://app.alt.xyz/browse/external-listing?id...,"$108,000",8,N/A,"Aug 24, 2023",252000.0,586500.0,342000.0,586500.0,1410000.0,239210.0,215285.00,29210.000000,0.877890
4,https://app.alt.xyz/browse/external-listing?id...,"$108,000",8,N/A,Auto 10,324000.0,586500.0,342000.0,586500.0,1410000.0,239210.0,215285.00,-30790.000000,1.128715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,https://app.alt.xyz/browse/external-listing?id...,"$15,600",4,N/A,Auto 10,38100.0,32101.0,32101.0,32101.0,38100.0,22280.7,20048.63,-9469.300000,1.425000
87,https://app.alt.xyz/browse/external-listing?id...,"$25,200",18,N/A,"Jul 21, 2022",50400.0,50400.0,50400.0,50400.0,50400.0,35090.0,31577.00,-6910.000000,1.196922
88,https://app.alt.xyz/browse/external-listing?id...,"$20,400",8,N/A,"Feb 20, 2018",7600.0,7600.0,7600.0,7600.0,7600.0,5130.0,4613.00,-1203.333333,1.234568
89,https://app.alt.xyz/browse/external-listing?id...,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
